# Null Value & Missing Data Analysis

## Census Data

- sa2_census.parquet
- sa2_pops.parquet
- sa2_to_postcode.parquet

There are 363 null postcodes in the sa2_to_postcode dataframe out of the 35,040 rows. This is likely due to there not being a postcode linked to those corresponding SA2 areas. It appears that the postcodes present in the consumer data are already covered in the SA2 to postcode coding index though so we can safely ignore these null postcodes.

There are no missing data entries in any of the initial sa2 datasets.

## Consumer Data
- consumer_fraud_probability.parquet
- consumer_tbl.parquet
- consumer_user_details.parquet

There are no null values or missing entries in the consumer dataframes.

## Merchant Data
- merchant_fraud_probability.parquet
- merchant_tbl.parquet

There are no null values or missing entries in the merchant dataframes.

## Transaction Data
- transactions_all.parquet

There are no null values or missing entries in the transaction dataframe.

## Joining Datasets

Joining datasets together, checking for null values after each join and missing data

### Join cons_tbl data with cons_user_det

There are no null values after joining consumer data. 
Also there are no missing data after the first join. That is the number of rows before the join is 499,999 and then there is still 499,999 rows after the join.

### Join cons_join with transaction_all

There are no null values after joining in the transaction data. 
Also there is no missing data after the first join. That is the number of rows in transaction before the join is 14,195,505 and then there is still 14,195,505 rows after the join.

### Join cons_transaction with cons_fp

There is 14,115,157 null values in the consumer fraud probability values out of the 14,195,717 rows. This is expected as not all transactions have been flagged as possibly being fraudulent, we will simply set the fraud probability to 0 for all other transactions.

Also there is an increase in the size of the data after this join. The number of rows before the join is 14,195,505 and then there 14,195,717 rows after the join. So 212 rows are gained in this join, most likely as a result of there being multiple matches in the right table (on user id and order datetime in the consumer_fraud_probability data) for a transaction in the left table. Due to the relatively small amount of duplicate matches compared to the total number of consumer fraud probability data, we can safely ignore these duplicate matches, especially when considering that the main use of the fraud probabilities will be for filtering out fraudulent transactions.

### Join cons_transaction_with_fraud with merch_tbl

Still the only column with null values is consumer fraud probabilities with 13,543,038 missing out of 13,614,854 rows.

There is a decrease in the size of this data after this join. The number of rows before the join is 14,195,505 whereas after the join there is 13,614,854 rows. This means that there are 580,651 missing rows of data after this join. This would likely be due to the fact that some of the Merchant ABNs in the transaction data are not covered by the merchant data. This could be due to an error with the merchant ABN entry in the transaction data or due to information relating to this merchant being missing from the merchant data. In either case there isn't much that can be done to resolve this and so we will simply ignore these transactions.

### Join cons_transaction_fraud_merchant with merch_fp

After joining the merchant fraud probability, there are 2 columns that have null values. These are consumer_fraud_probability_% and merchant_fraud_probability_%. consumer_fraud_probability_% has 13,543,038 null values and merchant_fraud_probability_% has 13,610,826 null values. This is again expected as not all transactions have been flagged as possibly being fraudulent, we will simply set the fraud probability to 0 for all other transactions.

There is also no change in the size of this data after this join. The number of rows before the join is 13,614,854 and remains the same after the join.

## Joining SA2/Census Data

### Join the sa2_to_post to sa2_pops

There are no null values from this join. Also no rows are missing, with 35,040 rows still remaining after the join

### Join the sa2_postcode_and_pops with sa2_census

There are still the null values present from the other previous search (363 postcode values are null). 

Also after the joining, there are still 35,040 rows in the dataframe. No rows have been lost joining the data together.

## Now to join the original data with the SA2 population and census data

No new columns with null values. Still consumer_fraud_probability_% and merchant_fraud_probability_%. consumer_fraud_probability_% has 11,312,983 nulls and merchant_fraud_probability_% has 11,369,563 null values.

There is a decrease in the size of this data after this join. The number of rows before the join is 13,614,854 whereas after the join there is 11,372,905 rows. This means that there are 2,241,949 missing rows of data after this join. This is likely due to the fact that not all postcodes present in the consumer data are present in the SA2 to postcode coding index. Upon further investigation online, it appears that a large portion of these postcodes either don't have SA2 codes for 2021 data or are tied to PO boxes. Therefore, we assume that it is safe to ignore the transaction data lost after joining with SA2 data as a large portion of the transactions, if not all, which were discarded were no longer considered part of SA2 data or had a PO box postcode entered for some reason.

# Outlier Analysis

For our outlier analysis we looked at the number of unique consumers that were below the 1% percentile (1 consumer) from our total dataset. In total, 45 records were removed from leaving 3864 merchants remaining. After conducting some outlier analysis, we noticed that these outliers had a higher average cost of order with smoe quite close to 2000. We determined that it was fit to remove these consumers as they would have had a sizeable amount of transactions above the assumed BNPL limit of 2000. This is because we wouldn't be able to profit off all of their transactions as consumers would have to use other payment methods for the transaction.

# Data Insights

For the the data inssights and aggreation we did the following:

1. Split the dataset from ETL into 5 groups being:
    1. Merchant 
    2. Consumers
    3. Orders
    4. Postcodes
    5. Descriptions



2. For Merchants:
    - Found that there are 4026 merchants

    - Found that all merchants each have only one take rate and revenue level.

    - Data was aggregated by merchant abn:
        1. Unique name, take rate, and revenue level was taken using the first function.

        2. Average was taken for the merchant fraud probability


3. For Consumers:
    - Data was aggregated by merchant abn:
        1. consumer_id was aggregated by counting the distinct number of consumer_ids for each merchant to give the unique number of consumers

        2. Average was taken for the consumer fraud probability

    - We created the following features:
        1. The number of repeat consumers was created by checking wheteher the number of order times was greater than 1.

        2. The average number of time consumers ordered was created by taking the average number of times a particular consumers ordered from a particular merchant.

    - There were 1456 merchants with no repeat consumers.


4. For Orders:
    - Data was aggregated by merchant abn:
        1. Order dates was aggregated by counting the distinct number of order dates for each merchant to give the number of orders.

        2. Average was taken for the dollar value of an order as average cost of order.

    - We created the following features:
        1. The average difference in consumers per month.

        2. The difference in consumer at the start of the transaction period and the end of the transaction period.

        3. The rate of growth of consumer for each merchant.

        4. The average dollar amount a consumer spent on a particular merchant.

        5. The revenue of a merchant.


5. For Postcodes:
    - Data was aggregated by merchant abn:
        1. Took the first value for merchant name as the merchant's name, as merchant name is unique.

        2. Count the distinct number of consumer postcodes to find the number of postcodes the merchant served. 

        3. Took the average of total weekly personal income.
    
        4. Took the average of total weekly family income.

        5. Took the average median age.

        6. Took the average household size.
    

    - We created the following features:
        1. The reach of merchant

        2. The average number of consumers that a merchant serves per postcode.

6. For Descriptions:
    - Each merchant has only one description
    - Data was aggregated by merchant abn:
        1. Merchant description was aggregation using the first function as it is unique for each merchant.

    - There are only 25 different merchant.
    - Merchants were put into 1 of 5 segments:
        1. Tech and Electronics
        
        2. Retail and Novelty
        
        3. Garden and Furnishings
        
        4. Antiques and Jewellery
        
        5. Specialized Services
    
7. Then we joined the aggregated datasets back together and created a new feature "bnpl_maximum_gain" which is the maximum amount the BNPL service can make off a merchant assuming all consumers use the BNPL service.

8. Insights from this dataset were visualized and the plots produced were added to the plot directory.

# Modeled Feature Engineering

We also created a modelled feature, which predicts the number of customers for a merchant in 3 years time, these are the steps we took:
- We checked the correlation between the number of customers and other features.

- We performed feature selection based on statistical tests.

- Split the data into test and training.

- Fitted 3 models to the data and determined which model performed the best by viewing residual vs fitted values graphs, R-Squared value and the RSME value.

- Identified XGBoost Regression model performed the best.

- Created new modelled feature for the number of customers in 3 years using the XGBoost Regression model.  


# Assumptions / Limitations

* Upon investigation we found that most BNPL providers provide a maximum credit limit of $2000. We therefore thought it would be reasonable to cap transactions in the data to $2000 to take this into account. 

* As mentioned in the Null & Missing Value Analysis section, we also discovered that the more up to date SA2 code to postcode index was missing a portion of the postcodes present in the consumer data which tended to be PO box postcodes or unique places such as Monash University for example. We ended up deciding that since many of these postcodes weren't typical places that the average consumer would live in, it would be safe to ignore any postcodes in the consumer data that couldn't be matched up with an SA2 area.

* We also assumed that any transactions that weren't provided a consumer or merchant fraud probability could safely be assumed to be 0% for these features.

# Ranking System

For the ranking system, we sectioned features into 6 different weights to help rank merchants for the the bnpl firm to partner with. The features given the most significant weight were expected profit, consumer growth and average spend per customer. Also important were the merchant and customer fraud probabilities. These were given negative weights. An overall score was taken from the weights to be used to rank the merchants, with a higher score preferable. It must be noted that the weights assigned weren't created perfectly. In future, the bnpl firm should be consulted to weight how they see fit.

An interesting finding is that the segment distribution for the top 100 merchants is different to the merchants as a whole. Most significantly different was that garden and furnishing merchants made up 32% of the top 100 merchants. Also it is interesting to note that retail and novelty merchants were expected to generate more profit than other segments inside the top 100.

# Future Improvements
These are some suggestions for going forward with this ranking system:

1. Developing More Modelled Features - We would like to create more modelled features aside from the one we created.

2. Incorportating more data - By incorporating more data we can better train the models and also improve the accuracy of the features in which we took the aggregated averages.

3. Consulting with the BNPL service about the weights of the Ranking System -  By consulting with the BNPL service, we can have a better understanding of what features the BNPL service values more and adjust the weights of these features accordingly, which would lead to better merchant partnerships.